### Importing Datasets and Models

In [ ]:
!pip -q install torchcam

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 721.5 kB/s eta 0:00:00


In [ ]:
!pip -q install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 292.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 5.5 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import shutil
import numpy as np
import pandas as pd

import seaborn as sns
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torchvision.transforms as transforms

from torchcam.methods import GradCAM

In [ ]:
!gdown -q 1g61gMabSflHdZzlQ5o6KUq5hpmIYZJI4 # Download PRECISE Hack Dataset
!unzip -q /content/PRECISE_Hackathon_Dataset.zip # Unzip Dataset
!rm -rf /content/PRECISE_Hackathon_Dataset.zip

In [ ]:
# Remove Masks, Prep Classification Dataset
data_path = '/content/PRECISE_Hackathon_Dataset'

for root, dir, img in os.walk(data_path):
  for image in img:
    if 'mask' in image:
      os.remove(os.path.join(root, image))

In [ ]:
!gdown -q 1XqyPeOKnoXX5e_BlgFkZqj4eGtSh1ixf # Download Swin Classification Model

In [ ]:
model = torch.load('/content/Breast-US-Model.pth', map_location=torch.device('cpu'), weights_only = False)

### Explainability Layer

In [ ]:
model.eval()
target_layer = model.features[7]
target_layer.requires_grad_(True)
cam_extractor_C = GradCAM(model, target_layer)

def preprocess_image(Input_Image):
  transform = transforms.Compose([
        transforms.Resize(320),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ])

  return transform(Input_Image).unsqueeze(0)

### Gradio Launcher

In [ ]:
def predict_and_visualize(Input_Image):
    input_tensor = preprocess_image(Input_Image)
    Class_Output = model(input_tensor)
    probabilities = torch.sigmoid(Class_Output)

    # Calculate confidence scores for all three classes
    confidence_benign = float(probabilities[0][0])
    confidence_malignant = float(probabilities[0][1])
    confidence_normal = float(probabilities[0][2])

    confidences = {
        "Benign": confidence_benign,
        "Malignant": confidence_malignant,
        "Normal": confidence_normal
    }

    # Get the predicted class
    predicted_class_idx = probabilities.argmax(dim=1).item()

    class_names = ["Benign", "Malignant", "Normal"]
    predicted_class = class_names[predicted_class_idx]

    # Generate activation map
    activation_map = cam_extractor_C(predicted_class_idx, Class_Output)

    # Process the image and heatmap
    img_array = np.array(Input_Image.convert('RGB'))
    img_array = cv2.resize(img_array, (224, 224))
    heatmap = activation_map[0].squeeze().numpy()
    heatmap = (heatmap - np.min(heatmap)) / (np.max(heatmap) - np.min(heatmap))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    heatmap = cv2.resize(heatmap, (img_array.shape[1], img_array.shape[0]))

    if img_array.shape == heatmap.shape:
        superimposed_img = cv2.addWeighted(img_array, 0.6, heatmap, 0.4, 0)
    else:
        raise ValueError("The dimensions or the number of channels of the images do not match.")

    superimposed_img = Image.fromarray(superimposed_img)

    return confidences, predicted_class, superimposed_img

In [ ]:
# Gradio Interface

import gradio as gr

examples = ['/content/PRECISE_Hackathon_Dataset/benign/benign (101).png',
            '/content/PRECISE_Hackathon_Dataset/benign/benign (17).png',
            '/content/PRECISE_Hackathon_Dataset/malignant/malignant (106).png',
            '/content/PRECISE_Hackathon_Dataset/malignant/malignant (168).png',
            '/content/PRECISE_Hackathon_Dataset/normal/normal (101).png',
            '/content/PRECISE_Hackathon_Dataset/normal/normal (17).png'
            ]

iface = gr.Interface(
    fn=predict_and_visualize,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Label(num_top_classes=3),
        gr.Image(label="Grad-CAM Visualisation", type="pil")
    ],

    title="Breast Cancer Classification with Grad-CAM Visualization",
    description="Upload an Image to Classify as Positive or Negative for Breast Cancer Proliferation.",
    examples = examples
)

iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://1ac3d1b3a0b67ec639.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://1ac3d1b3a0b67ec639.gradio.live
